# import

In [78]:
import pprint

from dotenv import load_dotenv

from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever

from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage



# loading env variables

In [79]:
load_dotenv()

True

# Langain & Ollama

## check ollama status

In [21]:
# !curl --location 'http://127.0.0.1:11434/api/generate' \
# --header 'Content-Type: application/json' \
# --data '{ \
#     "model": "llama3.2:3b", \
#     "prompt": "hello llama!",  \
#     "options": { \
#         "temperature": 0 \
#     } \
# }' \
# | python -m json.tool

curl: (3) URL rejected: Port number was not a decimal number between 0 and 65535
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0curl: (6) Could not resolve host: application
curl: (3) URL rejected: Port number was not a decimal number between 0 and 65535
curl: (3) URL rejected: Port number was not a decimal number between 0 and 65535
curl: (3) URL rejected: Port number was not a decimal number between 0 and 65535
curl: (3) URL rejected: Malformed input to a URL function
curl: (3) URL rejected: Port number was not a decimal number between 0 and 65535
curl: (3) URL rej

In [80]:
!curl http://localhost:11434/api/tags

{"models":[{"name":"llama3.3:70b","model":"llama3.3:70b","modified_at":"2024-12-08T16:10:08.2818115+01:00","size":42520413916,"digest":"a6eb4748fd2990ad2952b2335a95a7f952d1a06119a0aa6a2df6cd052a93a3fa","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"70.6B","quantization_level":"Q4_K_M"}},{"name":"llama3.1:8b-instruct-q8_0","model":"llama3.1:8b-instruct-q8_0","modified_at":"2024-10-19T17:43:51.1667287+02:00","size":8540789934,"digest":"b158ded76fa05be6bce8a682099ce5df8c5571340a04cf63a2923464679db576","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8.0B","quantization_level":"Q8_0"}},{"name":"mxbai-embed-large:335m","model":"mxbai-embed-large:335m","modified_at":"2024-10-19T17:31:12.9912311+02:00","size":669615493,"digest":"468836162de7f81e041c43663fedbbba921dcea9b9fefea135685a39b2d83dd8","details":{"parent_model":"","format":"gguf","family":"bert","families":["bert"],"parameter_size

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1720  100  1720    0     0   8131      0 --:--:-- --:--:-- --:--:--  8151


## Ollama model: configuration

In [81]:
llm = OllamaLLM(model="llama3.2:3b", temperature=0)
llm

OllamaLLM(model='llama3.2:3b', temperature=0.0)

## testing llm: invoke


In [83]:
# llm.invoke(input="tell me a joke")
response = llm.invoke("hello ollama!")

# response = llm.invoke("Create an agent that uses Ollama function calling in Langchain.")

print(response)

Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?


## testing llm: chat prompt template

In [84]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

chain = chat_prompt_template | llm

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

As a world-class technical documentation writer, I'd be happy to explain how Langsmith can support testing.

Langsmith is an AI-powered tool designed to generate high-quality, human-like content for various applications, including technical documentation. While its primary function is to create documentation, Langsmith can also assist with testing in several ways:

1. **Automated testing of generated content**: Langsmith's AI engine can be used to test the accuracy and completeness of generated content, such as user manuals, guides, or API documentation. By analyzing the output against a set of predefined criteria, Langsmith can identify potential errors or inconsistencies.
2. **Content validation**: Langsmith can validate the accuracy of existing content by comparing it with a knowledge graph or a database of known facts. This helps ensure that the generated content is up-to-date and accurate.
3. **Automated testing of documentation templates**: Langsmith's AI engine can be used to te

## testing llm: chat prompt template & StrOutputParser

In [85]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

output_parser = StrOutputParser()

chain = chat_prompt_template | llm | output_parser

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

As a technical documentation writer, I'd be happy to provide insights on how Langsmith can support testing.

Langsmith is an AI-powered tool designed for creating and managing technical documentation. While its primary focus is on documentation creation, it also offers some features that can indirectly aid in testing:

1. **Automated content analysis**: Langsmith's AI engine analyzes the generated content for accuracy, consistency, and relevance. This feature can help identify potential issues with the documentation, such as outdated information or inconsistencies in formatting.
2. **Content validation**: Langsmith's validation process checks the generated content against a set of predefined rules and standards. This ensures that the documentation meets certain quality and format requirements, which can be beneficial for testing purposes.
3. **Integration with testing frameworks**: Langsmith can integrate with popular testing frameworks like Selenium or Appium, allowing you to automate

## create vector store & a retriever

In [59]:
# 1. select a specfic datasource. In this case a web page. 
# 2. save extracted content from the web page as docs.
# 3. index the docs using FAISS vector store.
# 4. convert the vector store to retriever.

web_base_loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = web_base_loader.load()

# print(f"type(docs) : {type(docs)} \n")
# print(f"len(docs) : {len(docs)}\n")
# print(f"docs: {docs} \n")
# type(f"docs[0] : {docs[0]} \n")
# print(f"docs[0].page_content : {docs[0].page_content} \n")

recursive_character_text_splitter = RecursiveCharacterTextSplitter()
documents = recursive_character_text_splitter.split_documents(documents=docs)


# print(type(documents))
# print(len(documents))
# print(documents)
# print(documents[0])
# print(documents[2])

ollama_embedding = OllamaEmbeddings(model="llama3.2:3b")
vector_store = FAISS.from_documents(documents=documents, embedding=ollama_embedding)


# print(f"vector_store.index.ntotal: {vector_store.index.ntotal}")
# print(f"vector_store._get_retriever_tags() : {vector_store._get_retriever_tags()}")
# print(f"vector_store.index_to_docstore_id : {vector_store.index_to_docstore_id}") 
# print(f"type(vector_store.index_to_docstore_id) : {type(vector_store.index_to_docstore_id)}") 

vector_store_retriever = vector_store.as_retriever()
print(f"vector_store_retriever: {vector_store_retriever}")

vector_store_retriever: tags=['FAISS', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000204B820D220> search_kwargs={}


## document chain

In [64]:
# 5. create a chat prompt template
# 6. create a stuff document chain that accepts a llm model and chat prompt template & we can also run stuff document chain by passing in documents directly

chat_prompt_template = ChatPromptTemplate.from_template(
"""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

documents_chain = create_stuff_documents_chain(llm=llm, prompt=chat_prompt_template)
response = documents_chain.invoke(
    {
        "input": "how can langsmith help with testing?",
        "context": documents        
    }
)
print(response)

According to the provided context, LangSmith can help with testing in several ways:

1. Creating datasets: LangSmith allows developers to create collections of inputs and reference outputs, which can be used to run tests on their LLM applications.
2. Running custom evaluations: LangSmith makes it easy to run custom evaluations (both LLM and heuristic-based) to score test results.
3. Comparison view: LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application.
4. Playground environment: LangSmith's playground allows for rapid iteration and experimentation, enabling developers to quickly test out different prompts and models.
5. Beta testing: LangSmith enables beta testing, which involves collecting data on how the LLM application is performing in real-world scenarios, allowing developers to develop an understanding of the types of inputs the app is performing well or poorly on.
6. Capturing

## retrieval chain

In [71]:
# 7. create a document retrieval chain that takes vector store retriever and stuff document chain

retrieval_chain = create_retrieval_chain(vector_store_retriever, documents_chain) 
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

# print(type(response))
pprint.pprint(response, indent=4)

{   'answer': 'According to the provided context, LangSmith can help with '
              'testing in several ways:\n'
              '\n'
              '1. Creating datasets: LangSmith allows developers to create '
              'collections of inputs and reference outputs, which can be used '
              'to run tests on their LLM applications.\n'
              '2. Running custom evaluations: LangSmith makes it easy to run '
              'custom evaluations (both LLM and heuristic-based) to score test '
              'results.\n'
              '3. Comparison view: LangSmith provides a user-friendly '
              'comparison view for test runs to track and diagnose regressions '
              'in test scores across multiple revisions of the application.\n'
              '4. Playground environment: LangSmith provides a playground '
              'environment for rapid iteration and experimentation, allowing '
              'developers to quickly test out different prompts and model

## conversation retrieval chain

In [73]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

history_aware_retriever_chain = create_history_aware_retriever(llm, vector_store_retriever, chat_prompt_template)

In [76]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, chat_prompt_template)
retrieval_chain = create_retrieval_chain(history_aware_retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]

response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "tell me how"
})

pprint.pprint(response)

{'answer': 'Yes, LangSmith can definitely help you test your LLM applications. '
           "Here's a brief overview of the beta testing phase and some key "
           'features that support testing:\n'
           '\n'
           '1. **Beta Testing**: This is where developers collect more data on '
           'how their LLM applications perform in real-world scenarios.\n'
           '2. **Capturing Feedback**: You can gather human feedback on the '
           'responses produced by your application, which helps identify edge '
           'cases causing problematic responses.\n'
           '3. **Annotating Traces**: LangSmith allows you to send runs to '
           'annotation queues, where annotators can closely inspect '
           'interesting traces and annotate them with respect to different '
           'criteria.\n'
           '4. **Adding Runs to a Dataset**: You can add runs as examples to '
           'datasets, expanding your test coverage on real-world scenarios.\n'
       